<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2021/blob/main/2DHeatConductionPolar_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import newton_krylov
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates_default='plotly_dark'

In [2]:
rinner, router = 1., 4.
thetalo, thetahi = 0, np.pi/2
nr, ntheta = 10, 10
dr, dtheta = (router-rinner)/(nr-1), (thetahi-thetalo)/(ntheta-1)
theta, r = np.mgrid[thetalo:thetahi:ntheta*1j, rinner:router:nr*1j]
x,y = r*np.cos(theta), r*np.sin(theta)

In [3]:
Tinner, Touter = 130., 150.
Tthetalo, Tthetahi = 100., 150.
qk = 0

In [4]:
Tguess = np.full((nr,ntheta), np.mean(np.array([Tinner, Touter, Tthetalo, Tthetahi])))
d2r = np.zeros_like(Tguess)
d2theta = np.zeros_like(Tguess)

In [5]:
def hde(T):
    d2r[1:-1, :] = ((r[1:-1,:]+dr/2)*(T[2:,:] - T[1:-1,:]) - 
                    (r[1:-1,:]-dr/2)*(T[1:-1,:] - T[:-2,:]))/dr**2
    d2r[0,:]= ((r[0,:]+dr/2)*(T[1,:] - T[0,:]) - 
                    (r[0,:]-dr/2)*(T[0,:] - Tinner))/dr**2
    d2r[-1,:]=((r[-1,:]+dr/2)*(Touter - T[-1,:]) - 
                    (r[-1,:]-dr/2)*(T[-1,:] - T[-2,:]))/dr**2

    d2theta[:,1:-1] = (T[:,2:] - 2*T[:,1:-1] + T[:,:-2])/dtheta**2 /r[:,1:-1]
    d2theta[:,0]=(T[:,1] - 2*T[:,0] + Tthetalo)/dtheta**2/r[:,0]
    d2theta[:,-1]=(Tthetahi - 2*T[:,-1] + T[:,-2])/dtheta**2/r[:,-1]

    return d2r+d2theta+ qk*r

In [6]:
sol = newton_krylov(hde, Tguess)

In [7]:
fig = make_subplots()
fig.add_trace(go.Surface(x=x,y=y,z=sol))